### Paper 11-27-22

https://github.com/cnaimo/hybrid-ARIMA-LSTM-model


In [1]:
# import 

import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from pmdarima import auto_arima
import pmdarima as pm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
from talib import abstract
import json

In [2]:
def mean_absolute_percentage_error(actual, prediction):
    actual = pd.Series(actual)
    prediction = pd.Series(prediction)
    return 100 * np.mean(np.abs((actual - prediction))/actual)

In [3]:
def get_arima(data, train_len, test_len):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    train = data.head(train_len).values.tolist()
    test = data.tail(test_len).values.tolist()

    # Initialize model
    model = auto_arima(train, max_p=3, max_q=3, seasonal=False, trace=True,
                       error_action='ignore', suppress_warnings=True)

    # Determine model parameters
    model.fit(train)
    order = model.get_params()['order']
    print('ARIMA order:', order, '\n')

    # Genereate predictions
    prediction = []
    for i in range(len(test)):
        model = pm.ARIMA(order=order)
        model.fit(train)
        print('working on', i+1, 'of', test_len, '-- ' + str(int(100 * (i + 1) / test_len)) + '% complete')
        prediction.append(model.predict()[0])
        train.append(test[i])

    # Generate error data
    mse = mean_squared_error(test, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(pd.Series(test), pd.Series(prediction))
    return prediction, mse, rmse, mape



In [27]:
def get_lstm(data, train_len, test_len, lstm_len=4):
    # prepare train and test data
    data = data.tail(test_len + train_len).reset_index(drop=True)
    dataset = np.reshape(data.values, (len(data), 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset_scaled = scaler.fit_transform(dataset)
    x_train = []
    y_train = []
    x_test = []

    for i in range(lstm_len, train_len):
        x_train.append(dataset_scaled[i - lstm_len:i, 0])
        y_train.append(dataset_scaled[i, 0])
    for i in range(train_len, len(dataset_scaled)):
        x_test.append(dataset_scaled[i - lstm_len:i, 0])

    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Set up & fit LSTM RNN
    model = Sequential()
    model.add(LSTM(units=lstm_len, return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=int(lstm_len/2)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    model.fit(x_train, y_train, epochs=500, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate predictions
    prediction = model.predict(x_test)
    prediction = scaler.inverse_transform(prediction).tolist()

    output = []
    for i in range(len(prediction)):
        output.extend(prediction[i])
    prediction = output

    # Generate error data
    mse = mean_squared_error(data.tail(len(prediction)).values, prediction)
    rmse = mse ** 0.5
    mape = mean_absolute_percentage_error(data.tail(len(prediction)).reset_index(drop=True), pd.Series(prediction))
    return prediction, mse, rmse, mape


In [35]:
A=1
talib_moving_averages[A]
functions['KAMA'](data, int(optimized_period['KAMA']))
# type(int(optimized_period['KAMA']))
# functions[ma](data, optimized_period[ma])

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
          ...    
1495    75.547160
1496    75.602797
1497    75.548982
1498    75.482714
1499    75.430298
Length: 1500, dtype: float64

In [36]:
if __name__ == '__main__':
    # Load historical data
    # CSV should have columns: ['date', 'open', 'high', 'low', 'close', 'volume']
    data_file = 'C:/Users/Mya_L/Documents/JupyterLab_projects/Suplemnter/Data_for_paper-11-27-22/all_stocks_5yr.csv'
    data = pd.read_csv(data_file, index_col=0, header=0).tail(1500).reset_index(drop=True)

    # Initialize moving averages from Ta-Lib, store functions in dictionary
    talib_moving_averages = ['SMA', 'EMA', 'WMA', 'DEMA', 'KAMA', 'MIDPOINT', 'MIDPRICE', 'T3', 'TEMA', 'TRIMA']
    functions = {}
    for ma in talib_moving_averages:
        functions[ma] = abstract.Function(ma)

    # Determine kurtosis "K" values for MA period 4-99
    kurtosis_results = {'period': []}
    for i in range(4, 100):
        kurtosis_results['period'].append(i)
        for ma in talib_moving_averages:
            # Run moving average, remove last 252 days (used later for test data set), trim MA result to last 60 days
            ma_output = functions[ma](data[:-252], i).tail(60)

            # Determine kurtosis "K" value
            k = kurtosis(ma_output, fisher=False)

            # add to dictionary
            if ma not in kurtosis_results.keys():
                kurtosis_results[ma] = []
            kurtosis_results[ma].append(k)

    kurtosis_results = pd.DataFrame(kurtosis_results)
    kurtosis_results.to_csv('kurtosis_results.csv')

    # Determine period with K closest to 3 +/-5%
    optimized_period = {}
    for ma in talib_moving_averages:
        difference = np.abs(kurtosis_results[ma] - 3)
        df = pd.DataFrame({'difference': difference, 'period': kurtosis_results['period']})
        df = df.sort_values(by=['difference'], ascending=True).reset_index(drop=True)
        if df.at[0, 'difference'] < 3 * 0.05:
            optimized_period[ma] = df.at[0, 'period']
        else:
            print(ma + ' is not viable, best K greater or less than 3 +/-5%')

    print('\nOptimized periods:', optimized_period)

    simulation = {}
    for ma in optimized_period:
        # Split data into low volatility and high volatility time series
        low_vol = functions[ma](data, int(optimized_period[ma]))
        high_vol = data['close'] - low_vol

        # Generate ARIMA and LSTM predictions
        print('\nWorking on ' + ma + ' predictions')
        try:
            low_vol_prediction, low_vol_mse, low_vol_rmse, low_vol_mape = get_arima(low_vol, 1000, 252)
        except:
            print('ARIMA error, skipping to next MA type')
            continue

        high_vol_prediction, high_vol_mse, high_vol_rmse, high_vol_mape = get_lstm(high_vol, 1000, 252)

        final_prediction = pd.Series(low_vol_prediction) + pd.Series(high_vol_prediction)
        mse = mean_squared_error(final_prediction.values, data['close'].tail(252).values)
        rmse = mse ** 0.5
        mape = mean_absolute_percentage_error(data['close'].tail(252).reset_index(drop=True), final_prediction)

        # Generate prediction accuracy
        actual = data['close'].tail(252).values
        result_1 = []
        result_2 = []
        for i in range(1, len(final_prediction)):
            # Compare prediction to previous close price
            if final_prediction[i] > actual[i-1] and actual[i] > actual[i-1]:
                result_1.append(1)
            elif final_prediction[i] < actual[i-1] and actual[i] < actual[i-1]:
                result_1.append(1)
            else:
                result_1.append(0)

            # Compare prediction to previous prediction
            if final_prediction[i] > final_prediction[i-1] and actual[i] > actual[i-1]:
                result_2.append(1)
            elif final_prediction[i] < final_prediction[i-1] and actual[i] < actual[i-1]:
                result_2.append(1)
            else:
                result_2.append(0)

        accuracy_1 = np.mean(result_1)
        accuracy_2 = np.mean(result_2)

        simulation[ma] = {'low_vol': {'prediction': low_vol_prediction, 'mse': low_vol_mse,
                                      'rmse': low_vol_rmse, 'mape': low_vol_mape},
                          'high_vol': {'prediction': high_vol_prediction, 'mse': high_vol_mse,
                                       'rmse': high_vol_rmse},
                          'final': {'prediction': final_prediction.values.tolist(), 'mse': mse,
                                    'rmse': rmse, 'mape': mape},
                          'accuracy': {'prediction vs close': accuracy_1, 'prediction vs prediction': accuracy_2}}

        # save simulation data here as checkpoint
        with open('simulation_data.json', 'w') as fp:
            json.dump(simulation, fp)

    for ma in simulation.keys():
        print('\n' + ma)
        print('Prediction vs Close:\t\t' + str(round(100*simulation[ma]['accuracy']['prediction vs close'], 2))
              + '% Accuracy')
        print('Prediction vs Prediction:\t' + str(round(100*simulation[ma]['accuracy']['prediction vs prediction'], 2))
              + '% Accuracy')
        print('MSE:\t', simulation[ma]['final']['mse'],
              '\nRMSE:\t', simulation[ma]['final']['rmse'],
              '\nMAPE:\t', simulation[ma]['final']['mape'])

EMA is not viable, best K greater or less than 3 +/-5%
WMA is not viable, best K greater or less than 3 +/-5%
DEMA is not viable, best K greater or less than 3 +/-5%
TEMA is not viable, best K greater or less than 3 +/-5%

Optimized periods: {'SMA': 53, 'KAMA': 95, 'MIDPOINT': 81, 'MIDPRICE': 54, 'T3': 82, 'TRIMA': 72}

Working on SMA predictions
Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=-4836.223, Time=1.83 sec
 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=-4828.276, Time=0.30 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=-4832.699, Time=0.35 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=-4833.726, Time=0.88 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=-4829.783, Time=0.08 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=-4834.591, Time=0.70 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=-4834.538, Time=0.60 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=-4834.525, Time=1.74 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=-4837.127, Time=1.83 sec
 ARIM

C:\Users\Mya_L\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


working on 1 of 252 -- 0% complete


C:\Users\Mya_L\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


working on 2 of 252 -- 0% complete


...

working on 238 of 252 -- 94% complete
working on 239 of 252 -- 94% complete


C:\Users\Mya_L\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


working on 240 of 252 -- 95% complete
working on 241 of 252 -- 95% complete
working on 242 of 252 -- 96% complete
working on 243 of 252 -- 96% complete
working on 244 of 252 -- 96% complete


C:\Users\Mya_L\AppData\Roaming\Python\Python39\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


working on 245 of 252 -- 97% complete
working on 246 of 252 -- 97% complete
working on 247 of 252 -- 98% complete
working on 248 of 252 -- 98% complete
working on 249 of 252 -- 98% complete
working on 250 of 252 -- 99% complete
working on 251 of 252 -- 99% complete
working on 252 of 252 -- 100% complete
Epoch 1/500
996/996 - 24s - loss: 0.0151 - 24s/epoch - 24ms/step
Epoch 2/500
996/996 - 6s - loss: 0.0057 - 6s/epoch - 6ms/step
Epoch 3/500
996/996 - 6s - loss: 0.0032 - 6s/epoch - 6ms/step
Epoch 4/500
996/996 - 6s - loss: 0.0025 - 6s/epoch - 6ms/step
Epoch 5/500
996/996 - 6s - loss: 0.0021 - 6s/epoch - 6ms/step
Epoch 6/500
996/996 - 5s - loss: 0.0016 - 5s/epoch - 5ms/step
Epoch 7/500
996/996 - 5s - loss: 0.0014 - 5s/epoch - 5ms/step
Epoch 8/500
996/996 - 5s - loss: 0.0012 - 5s/epoch - 5ms/step
Epoch 9/500
996/996 - 4s - loss: 0.0011 - 4s/epoch - 4ms/step
Epoch 10/500
996/996 - 10s - loss: 0.0010 - 10s/epoch - 10ms/step
Epoch 11/500
996/996 - 6s - loss: 9.7328e-04 - 6s/epoch - 7ms/step
E

In [29]:
data.head(10)

,open,high,low,close,volume,Name
0,44.18,44.5500,44.000,44.55,3605293,ZION
1,44.65,45.2400,44.510,45.11,2298903,ZION
2,44.93,45.1600,44.700,44.90,2485994,ZION
3,48.33,48.3300,45.950,46.53,3320662,ZION
4,46.67,46.6899,44.960,44.96,2925913,ZION
5,44.95,45.5700,44.750,45.36,2686381,ZION
6,45.03,45.4300,44.689,45.33,1706120,ZION
7,45.40,45.6750,44.970,45.28,1738722,ZION
8,45.84,46.3000,45.155,45.26,3154365,ZION
9,45.35,45.7600,45.030,45.28,1963227,ZION
